In [ ]:
import pandas as pd
import numpy as np
data=pd.read_csv("origindata.csv")
print(data.index)
%matplot inline

In [ ]:
#Index(['flight', 'name', 'date', 'time'], dtype='object')
name_index=set(data["name"].values)
date_index=set(data["date"].values)
flight_index=set(data["flight"].values)
merged_data={}
merged_data["flight"]=[]
merged_data["date"]=[]
for i in name_index:
    merged_data[i]=[]
merged_data

In [ ]:
#处理数据，将航班和字段提取出来得到各个保障节点的数据
index=1
for date in date_index:
    index+=1
    data_temp=data[data["date"]==date]
    flight_set=set(data_temp["flight"].values)
    for flight_temp in flight_set:
        merged_data["date"].append(date) #增加航班日期
        merged_data["flight"].append(flight_temp) #增加航班号
        node_value_all=data_temp[data_temp["flight"]==flight_temp]
        for node_name in name_index:
            try:
                node_value=node_value_all[node_value_all["name"]==node_name]["time"].values()
                print(node_value)
                merged_data[node_name].append(node_value)
            except:
                merged_data[node_name].append(-1)
    if index>10:
        break